# Import required modules

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import warnings
import pickle

sys.path.append(os.path.abspath(os.path.join('../scripts')))

from clean_dataframe import Utility
from plot_dataframe import Plotter

plot = Plotter()
cleaner = Utility()

In [29]:
warnings.filterwarnings('ignore')

# Read Data

In [30]:
df = pd.read_csv("../data/clean_data.csv")

In [31]:
# select only relevant columns for exprience analysis
tellco_exprience_df = df[['MSISDN/Number','Avg RTT DL (ms)','Avg RTT UL (ms)','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)','TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)','Handset Type']]
tellco_exprience_df

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type
0,33659219748,97.0,7.0,3845.0,2535.0,3231397.0,7230.0,Samsung Galaxy Note 8 (Sm-N950F Ds)
1,33664473872,213.5,4.0,28305.0,6500.0,5722628.0,222238.0,Huawei B593S-22
2,33659219748,79.0,4.0,4148.0,4540.0,13165244.0,34638.0,Samsung Galaxy Note 8 (Sm-N950F Ds)
3,33753758738,39.0,33.0,263.0,620.0,12964929.0,222238.0,Samsung Galaxy J3 (Sm-J330)
4,33658752999,213.5,43.0,29501.0,10557.0,9844005.0,21586.0,Huawei E5180
...,...,...,...,...,...,...,...,...
49504,33665236895,70.0,20.0,46545.0,1575.0,8414820.0,50779.0,Huawei B528S-23A
49505,33666584437,213.5,61.0,9150.0,604.0,410605.0,5837.0,Samsung Galaxy A5 Sm-A520F
49506,33685838753,69.0,6.0,10960.0,877.0,83246.0,9136.0,Samsung Galaxy A8 (2018)
49507,33761274518,55.0,3.0,30741.0,12623.0,2288.0,2770.0,Apple iPhone 6S (A1688)


In [32]:
# merge Avg RTT (ms) , Avg Bearer TP (kbps) and Total TCP Retrans. Vol (Bytes)
tellco_exprience_df['Total Avg RTT (ms)'] = tellco_exprience_df['Avg RTT DL (ms)'] + tellco_exprience_df['Avg RTT UL (ms)']
tellco_exprience_df['Total Avg Bearer TP (kbps)'] = tellco_exprience_df['Avg Bearer TP DL (kbps)'] + tellco_exprience_df['Avg Bearer TP UL (kbps)']
tellco_exprience_df['Total TCP Retrans. Vol (Bytes)'] = tellco_exprience_df['TCP DL Retrans. Vol (Bytes)'] + tellco_exprience_df['TCP UL Retrans. Vol (Bytes)']
tellco_exprience_df.head()

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,33659219748,97.0,7.0,3845.0,2535.0,3231397.0,7230.0,Samsung Galaxy Note 8 (Sm-N950F Ds),104.0,6380.0,3238627.0
1,33664473872,213.5,4.0,28305.0,6500.0,5722628.0,222238.0,Huawei B593S-22,217.5,34805.0,5944866.0
2,33659219748,79.0,4.0,4148.0,4540.0,13165244.0,34638.0,Samsung Galaxy Note 8 (Sm-N950F Ds),83.0,8688.0,13199882.0
3,33753758738,39.0,33.0,263.0,620.0,12964929.0,222238.0,Samsung Galaxy J3 (Sm-J330),72.0,883.0,13187167.0
4,33658752999,213.5,43.0,29501.0,10557.0,9844005.0,21586.0,Huawei E5180,256.5,40058.0,9865591.0


In [33]:
tellco_exprience_df = tellco_exprience_df[['MSISDN/Number','Total Avg RTT (ms)','Total Avg Bearer TP (kbps)','Total TCP Retrans. Vol (Bytes)','Handset Type']]
tellco_exprience_df.head()

,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
0,33659219748,104.0,6380.0,3238627.0,Samsung Galaxy Note 8 (Sm-N950F Ds)
1,33664473872,217.5,34805.0,5944866.0,Huawei B593S-22
2,33659219748,83.0,8688.0,13199882.0,Samsung Galaxy Note 8 (Sm-N950F Ds)
3,33753758738,72.0,883.0,13187167.0,Samsung Galaxy J3 (Sm-J330)
4,33658752999,256.5,40058.0,9865591.0,Huawei E5180


In [34]:
# tellco_exprience_df = tellco_exprience_df.copy()
tellco_exprience_df1 = tellco_exprience_df.groupby(
    'MSISDN/Number').agg({'Total Avg RTT (ms)': 'sum', 'Total Avg Bearer TP (kbps)': 'sum', 'Total TCP Retrans. Vol (Bytes)': 'sum','Handset Type': [lambda x: x.mode()[0]] }) #' '.join(x)
tellco_exprience_df1

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
,sum,sum,sum,<lambda>
MSISDN/Number,,,,
33601008617,91.0,52995.0,9370832.0,Apple iPhone Se (A1723)
33601011634,39.0,42416.0,110232.0,Huawei Mate 10 Pro Porsche Design Huawei Mate 10
33601021217,160.0,19256.0,13171894.0,Apple iPhone 7 Plus (A1784)
33601031129,60.0,38190.0,2325497.0,Apple iPhone 8 Plus (A1897)
33601034530,217.5,8539.0,2006261.0,Apple iPhone 7 (A1778)
...,...,...,...,...
33789914536,70.0,554.0,2734889.0,Apple iPhone Se (A1723)
33789922012,128.0,43305.0,1229545.0,Samsung Galaxy S7 Edge (Sm-G935X)


In [35]:
tellco_exprience_df = pd.DataFrame(columns=[
    "Total Avg RTT (ms)",
    "Total Avg Bearer TP (kbps)",
    "Total TCP Retrans. Vol (Bytes)",
    "Handset Type"])

tellco_exprience_df["Total Avg RTT (ms)"] = tellco_exprience_df1["Total Avg RTT (ms)"]['sum']
tellco_exprience_df["Total Avg Bearer TP (kbps)"] = tellco_exprience_df1["Total Avg Bearer TP (kbps)"]['sum']
tellco_exprience_df["Total TCP Retrans. Vol (Bytes)"] = tellco_exprience_df1["Total TCP Retrans. Vol (Bytes)"]['sum']
tellco_exprience_df["Handset Type"] = tellco_exprience_df1["Handset Type"]['<lambda>']
tellco_exprience_df.head()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
MSISDN/Number,,,,
33601008617,91.0,52995.0,9370832.0,Apple iPhone Se (A1723)
33601011634,39.0,42416.0,110232.0,Huawei Mate 10 Pro Porsche Design Huawei Mate 10
33601021217,160.0,19256.0,13171894.0,Apple iPhone 7 Plus (A1784)
33601031129,60.0,38190.0,2325497.0,Apple iPhone 8 Plus (A1897)
33601034530,217.5,8539.0,2006261.0,Apple iPhone 7 (A1778)


# Top 10 of the top, bottom and most frequent Datas

In [36]:
rtt = tellco_exprience_df.sort_values('Total Avg RTT (ms)', ascending=False)
rtt.head(10)['Total Avg RTT (ms)']

MSISDN/Number
33699231421    1546.5
33661827479    1404.0
33658263267    1352.0
33699168714    1323.5
33664709594    1308.5
33761249975    1234.0
33662014376    1226.0
33665218470    1213.5
33760112066    1206.0
33668563241    1195.0
Name: Total Avg RTT (ms), dtype: float64

In [37]:
tellco_exprience_df['Total Avg RTT (ms)'].value_counts().head(10)

49.0     467
52.0     440
46.0     429
48.0     429
44.0     426
55.0     425
216.5    425
50.0     416
51.0     405
217.5    404
Name: Total Avg RTT (ms), dtype: int64

In [38]:
br_tp = tellco_exprience_df.sort_values('Total Avg Bearer TP (kbps)', ascending=False)
br_tp

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
MSISDN/Number,,,,
33659546392,512.0,650628.5,27712733.0,Huawei B528S-23A
33762644658,1028.0,502403.5,66762110.0,Huawei B528S-23A
33699248832,549.0,501263.5,75939773.0,Huawei B528S-23A
33666461685,543.0,470459.0,12325061.0,Huawei B528S-23A
33658727547,525.0,465453.0,8993383.0,Huawei B528S-23A
...,...,...,...,...
33658615582,229.5,57.0,8678.0,Apple iPhone 6S (A1688)
33621826880,233.5,53.0,14867.0,Apple iPhone Se (A1723)
33763459790,228.5,53.0,23200.0,Apple iPhone Xr (A2105)


In [39]:
br_tp.head(10)['Total Avg Bearer TP (kbps)']

MSISDN/Number
33659546392    650628.5
33762644658    502403.5
33699248832    501263.5
33666461685    470459.0
33658727547    465453.0
33660503175    449840.5
33668708263    446830.0
33763862031    446161.5
33699327554    435707.0
33698935800    435185.0
Name: Total Avg Bearer TP (kbps), dtype: float64

In [40]:
br_tp.tail(10)['Total Avg Bearer TP (kbps)']

MSISDN/Number
33618145282    60.0
33650717329    59.0
33646320432    58.0
33686839010    57.0
33661835953    57.0
33658615582    57.0
33621826880    53.0
33763459790    53.0
33669105392    52.0
33661466916    38.0
Name: Total Avg Bearer TP (kbps), dtype: float64

In [41]:
tellco_exprience_df['Total Avg Bearer TP (kbps)'].value_counts().head(10)

111707.5    368
223415.0     14
255.0         7
356.0         7
665.0         7
362.0         7
460.0         7
501.0         7
522.0         6
607.0         6
Name: Total Avg Bearer TP (kbps), dtype: int64

In [42]:
tcp_retransmitted = tellco_exprience_df.sort_values('Total TCP Retrans. Vol (Bytes)', ascending=False)
tcp_retransmitted

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
MSISDN/Number,,,,
33699231421,1546.5,150767.0,80324892.0,Huawei B528S-23A
33760112066,1206.0,206536.0,80036872.0,Huawei B528S-23A
33763730582,596.0,355358.0,79340500.0,Huawei B528S-23A
33658162536,744.0,286086.0,79264308.0,Huawei B528S-23A
33663654533,733.0,236073.0,79087167.0,Huawei B528S-23A
...,...,...,...,...
33762062356,41.0,32577.0,129.0,Apple iPhone 8 (A1905)
33614777138,64.0,8654.0,128.0,Huawei Ascend Mate7
33641080629,81.0,251.0,113.0,Apple iPhone 6 (A1549)


In [43]:
tcp_retransmitted.head(10)['Total TCP Retrans. Vol (Bytes)']

MSISDN/Number
33699231421    80324892.0
33760112066    80036872.0
33763730582    79340500.0
33658162536    79264308.0
33663654533    79087167.0
33665639832    76126144.0
33699248832    75939773.0
33665778430    75568629.0
33666819782    73600424.0
33761214656    71789757.0
Name: Total TCP Retrans. Vol (Bytes), dtype: float64

In [44]:
tcp_retransmitted.tail(10)['Total TCP Retrans. Vol (Bytes)']

MSISDN/Number
33650681239    176.0
33651768427    175.0
33607833304    143.0
33626357513    134.0
33768257339    134.0
33762062356    129.0
33614777138    128.0
33641080629    113.0
33659400378    108.0
33650128412     97.0
Name: Total TCP Retrans. Vol (Bytes), dtype: float64

In [45]:
tellco_exprience_df['Total TCP Retrans. Vol (Bytes)'].value_counts().head(10)

13387482.0    1404
26774964.0     106
13166574.0      22
2660.0          22
5320.0          21
6650.0          18
40162446.0      17
3990.0          17
7980.0          13
53549928.0      11
Name: Total TCP Retrans. Vol (Bytes), dtype: int64

In [46]:
tellco_exprience_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38911 entries, 33601008617 to 33789996170
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Total Avg RTT (ms)              38911 non-null  float64
 1   Total Avg Bearer TP (kbps)      38911 non-null  float64
 2   Total TCP Retrans. Vol (Bytes)  38911 non-null  float64
 3   Handset Type                    38911 non-null  object 
dtypes: float64(3), object(1)
memory usage: 1.5+ MB


# Handset Type Analysis

In [47]:
handset_type_exprience_df = tellco_exprience_df.groupby('Handset Type').agg({'Total Avg Bearer TP (kbps)': 'mean', 'Total TCP Retrans. Vol (Bytes)': 'mean','Total Avg RTT (ms)': "mean"}) #.mean()
handset_type_exprience_df #.sort_values(by='Total Avg Bearer TP (kbps)', ascending=False).head()

,Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Total Avg RTT (ms)
Handset Type,,,
A-Link Telecom I. Cubot Note Plus,6699.000000,6.162130e+05,221.500000
A-Link Telecom I. Cubot Note S,8937.000000,1.323250e+07,217.500000
A-Link Telecom I. Cubot Nova,56217.000000,1.371700e+05,44.000000
A-Link Telecom I. Cubot Power,69468.000000,8.041000e+03,43.000000
A-Link Telecom I. Cubot R9,4039.000000,3.099900e+04,231.000000
...,...,...,...
Zte Mf626,6096.000000,4.852921e+06,220.500000
Zte Racer Iii Mini Zte Switch X1 Bouygues Telecom Bs 402 Blade Q Mini Zte Blade G Pro,3634.000000,2.292400e+04,184.000000
Zyxel Communicat. Lte7460,52364.000000,1.338748e+07,133.000000


In [48]:
#average TCP retransmission view per handset type
handset_type_exprience_df.sort_values('Total TCP Retrans. Vol (Bytes)', ascending=False).head()

,Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Total Avg RTT (ms)
Handset Type,,,
Sierra Wireless Usb305,33665.000000,2.677496e+07,249.000000
Tp-Link Technolo. Tl-Mr6400,130959.666667,1.849000e+07,430.833333
Huawei E5776S-32,38304.666667,1.511637e+07,199.666667
Xiaomi Communica. Xiaomi Note,38276.000000,1.399123e+07,208.000000
Samsung Galaxy Tab A (Sm-T585),19983.500000,1.340981e+07,263.000000


# k-means clustering

In [49]:
tellco_exprience_df.drop(['Handset Type'], axis=1, inplace=True)

In [50]:

scale_data = StandardScaler().fit_transform(tellco_exprience_df)
scale_data

array([[-0.37139792,  0.09535923,  0.59768534],
       [-0.78096612, -0.11760067, -0.56884   ],
       [ 0.17206758, -0.58382161,  1.07649181],
       ...,
       [-0.67857407,  1.24364974, -0.22951498],
       [-0.54467678,  1.12858416, -0.58145378],
       [-0.67857407, -0.76280134, -0.55525918]])

In [51]:
normalized_data = normalize(scale_data)
normalized_data

array([[-0.52301393,  0.13428779,  0.84167881],
       [-0.80238821, -0.12082648, -0.58444342],
       [ 0.1391407 , -0.47210142,  0.87049418],
       ...,
       [-0.47280739,  0.86653294, -0.15991825],
       [-0.3942729 ,  0.8169435 , -0.42089452],
       [-0.58388488, -0.65635896, -0.47777752]])

In [52]:
kmeans = KMeans(n_clusters = 3, random_state = 42).fit(normalized_data)
y_kmeans = kmeans.fit_predict(normalized_data)
X = np.array(normalized_data)
y_kmeans

array([1, 0, 1, ..., 1, 0, 0])

In [53]:
clustered_tellco_exprience_df = tellco_exprience_df.copy()
clustered_tellco_exprience_df.insert(0, 'Cluster', y_kmeans)
clustered_tellco_exprience_df

,Cluster,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
MSISDN/Number,,,,
33601008617,1,91.0,52995.0,9370832.0
33601011634,0,39.0,42416.0,110232.0
33601021217,1,160.0,19256.0,13171894.0
33601031129,0,60.0,38190.0,2325497.0
33601034530,2,217.5,8539.0,2006261.0
...,...,...,...,...
33789914536,0,70.0,554.0,2734889.0
33789922012,0,128.0,43305.0,1229545.0
33789942399,1,52.0,110037.5,2804004.0


# segment users into groups of experiences

In [54]:
clustered_tellco_exprience_df[clustered_tellco_exprience_df["Cluster"]==0][["Total Avg RTT (ms)","Total Avg Bearer TP (kbps)","Total TCP Retrans. Vol (Bytes)"]].describe()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
count,20025.000000,20025.000000,2.002500e+04
mean,65.454981,38326.218202,9.212037e+05
std,24.953624,26205.137322,1.427456e+06
min,19.000000,66.000000,9.700000e+01
25%,46.000000,18049.000000,4.283500e+04
50%,60.000000,34327.000000,2.657500e+05
75%,81.000000,53887.000000,1.171816e+06
max,174.000000,117261.000000,1.107263e+07


In [55]:
clustered_tellco_exprience_df[clustered_tellco_exprience_df["Cluster"]==1][["Total Avg RTT (ms)","Total Avg Bearer TP (kbps)","Total TCP Retrans. Vol (Bytes)"]].describe()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
count,10895.000000,10895.000000,1.089500e+04
mean,220.970766,90112.716797,1.356265e+07
std,181.722365,68376.029963,1.032539e+07
min,26.000000,364.000000,2.728000e+03
25%,96.000000,43177.500000,7.488420e+06
50%,165.000000,75905.000000,1.318877e+07
75%,269.000000,111707.500000,1.338748e+07
max,1546.500000,650628.500000,8.032489e+07


In [56]:
clustered_tellco_exprience_df[clustered_tellco_exprience_df["Cluster"]==2][["Total Avg RTT (ms)","Total Avg Bearer TP (kbps)","Total TCP Retrans. Vol (Bytes)"]].describe()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
count,7991.000000,7991.000000,7.991000e+03
mean,207.419534,16081.073958,1.725874e+06
std,67.226557,15832.811900,2.175257e+06
min,86.000000,38.000000,1.820000e+02
25%,165.000000,3679.000000,1.498365e+05
50%,216.500000,11208.000000,7.985200e+05
75%,224.500000,23552.500000,2.554711e+06
max,1105.500000,106160.000000,1.473906e+07


In [57]:
# save the data
clustered_tellco_exprience_df.to_csv('../data/user_experience_data.csv')

In [58]:
with open("../models/user_experience.pkl", "wb") as f:
    pickle.dump(kmeans, f)